<a href="https://colab.research.google.com/github/ToonToonToon/229352-Statical-Learning-For-DataScience-2/blob/main/Lab01_Data_Preprocessingg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve

# For Fashion-MNIST
from tensorflow.keras.datasets import fashion_mnist

# For 20 Newsgroups
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## Part 1: Marketing Campaign Dataset - Manual Data Preprocessing & Logistic Regression

### Load the Marketing Campaign Dataset ([Data Information](https://archive.ics.uci.edu/dataset/222/bank+marketing))

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be (`'yes'`) or not (`'no'`) subscribed.

In [23]:
bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'

df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx",	"euribor3m", "nr.employed"], axis=1)
print("Shape of the dataset:", df.shape)
df.head()

Shape of the dataset: (41188, 16)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no


### Data Exploration

In [24]:
print("--- Missing Values Count ---")
print(df.isnull().sum())

--- Missing Values Count ---
age               0
job             330
marital          80
education      1731
default        8597
housing         990
loan            990
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
y                 0
dtype: int64


In [25]:
print("--- Unique Values for Categorical Columns ---")
for col in df.select_dtypes(include='object').columns:
    print(f"\n'{col}' unique values:")
    print(df[col].value_counts(dropna=False)) # Include NaN counts

--- Unique Values for Categorical Columns ---

'job' unique values:
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
NaN                330
Name: count, dtype: int64

'marital' unique values:
marital
married     24928
single      11568
divorced     4612
NaN            80
Name: count, dtype: int64

'education' unique values:
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
NaN                     1731
illiterate                18
Name: count, dtype: int64

'default' unique values:
default
no     32588
NaN     8597
yes        3
Name: count, dtype: int64

'housing' unique values:
housing
yes    21576
no     18622
NaN      990
Name: count, dtype: int64


### Data Preprocessing

In [26]:
# Map target variable 'y' to 0 (no) and 1 (yes)
df['y_new'] = df['y'].map({'yes':1, 'no':0}) # Write your code here


# Drop 'duration' due to data leakage


# Define features (X) and target (y)
y = df['y_new']
x = df.drop(['y', 'y_new'], axis=1)
# Split the data BEFORE any transformations


# Print data shape
print(x)
print(y)


       age          job  marital            education default housing loan    contact month day_of_week  duration  campaign  pdays  previous     poutcome
0       56    housemaid  married             basic.4y      no      no   no  telephone   may         mon       261         1    999         0  nonexistent
1       57     services  married          high.school     NaN      no   no  telephone   may         mon       149         1    999         0  nonexistent
2       37     services  married          high.school      no     yes   no  telephone   may         mon       226         1    999         0  nonexistent
3       40       admin.  married             basic.6y      no      no   no  telephone   may         mon       151         1    999         0  nonexistent
4       56     services  married          high.school      no      no  yes  telephone   may         mon       307         1    999         0  nonexistent
...    ...          ...      ...                  ...     ...     ...  ...  

In [27]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
X_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
1034,44,management,married,university.degree,no,no,no,telephone,may,wed,51,1,999,0,nonexistent
29476,43,technician,divorced,professional.course,NaN,no,no,cellular,apr,mon,131,2,999,1,failure
26872,35,management,single,NaN,no,yes,yes,cellular,nov,thu,207,1,999,0,nonexistent
14287,26,admin.,single,high.school,no,yes,no,cellular,jul,mon,115,2,999,0,nonexistent
35157,39,technician,divorced,high.school,no,yes,no,telephone,may,fri,15,11,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41108,64,NaN,married,NaN,no,no,no,cellular,nov,mon,78,1,3,3,success
7353,36,blue-collar,single,basic.9y,NaN,no,no,telephone,may,fri,45,3,999,0,nonexistent
26400,31,admin.,single,university.degree,no,yes,no,cellular,nov,thu,114,1,999,0,nonexistent
36979,46,admin.,divorced,high.school,no,yes,no,telephone,jul,thu,176,1,999,0,nonexistent


We will apply `StandardScaler()`, `OrdinalEncoder()`, and `OneHotEncoder()` on a few selected columns.

**1. Numerical Feature: `age` and `campaign` (Standard Scaling)**

In [7]:
num_cols_demo = ['age', 'campaign']

scaler = StandardScaler()

# Fit the scaler ONLY on the training da

X_train[num_cols_demo] = scaler.fit_transform(X_train[num_cols_demo])
X_test[num_cols_demo] = scaler.transform(X_test[num_cols_demo])



Let's take a look at the transformed `age` and `campaign` features and their statistics.

In [8]:
X_train.describe()

,age,duration,campaign,pdays,previous
count,2.883100e+04,28831.000000,2.883100e+04,28831.000000,28831.000000
mean,1.217468e-16,256.515279,1.478706e-18,963.042836,0.172349
std,1.000017e+00,260.749733,1.000017e+00,185.509803,0.492630
min,-2.213834e+00,0.000000,-5.595746e-01,0.000000,0.000000
25%,-7.708606e-01,101.000000,-5.595746e-01,999.000000,0.000000
50%,-1.936711e-01,178.000000,-1.978947e-01,999.000000,0.000000
75%,6.721131e-01,317.000000,1.637853e-01,999.000000,0.000000
max,5.578224e+00,4918.000000,1.933282e+01,999.000000,7.000000


**2. Ordinal Feature: `education` (Ordinal Encoding with Imputation)**

- **Imputation**

In [9]:
ord_col_demo = ['education']

imputer_ord = SimpleImputer(strategy='most_frequent')

X_train[ord_col_demo] = imputer_ord.fit_transform(X_train[ord_col_demo])
X_test[ord_col_demo] = imputer_ord.transform(X_test[ord_col_demo])

X_train['education']

,education
28140,professional.course
3875,university.degree
13259,professional.course
34570,high.school
3956,university.degree
...,...
5763,university.degree
26748,university.degree
25202,professional.course
8912,basic.9y


- **Ordinal Encoding**

In [10]:
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

In [11]:
ordinal_encoder = OrdinalEncoder(categories=[education_categories])

X_train[ord_col_demo] = ordinal_encoder.fit_transform(X_train[ord_col_demo])
X_test[ord_col_demo] = ordinal_encoder.transform(X_test[ord_col_demo])

X_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
2956,-0.674662,blue-collar,married,4.0,no,yes,no,telephone,may,wed,183,0.525465,999,0,nonexistent
32506,-0.386068,entrepreneur,divorced,2.0,no,yes,no,cellular,may,fri,53,0.525465,999,0,nonexistent
24521,-0.482266,self-employed,single,5.0,no,yes,no,cellular,nov,mon,128,-0.197895,999,0,nonexistent
7681,-0.963257,blue-collar,single,2.0,no,yes,no,telephone,may,fri,72,0.163785,999,0,nonexistent
14626,1.634096,blue-collar,divorced,2.0,NaN,yes,yes,cellular,jul,tue,329,1.248825,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20404,-0.482266,services,married,4.0,no,no,no,cellular,aug,tue,63,-0.559575,999,0,nonexistent
26232,0.383518,technician,single,6.0,no,no,no,cellular,nov,thu,284,-0.559575,999,0,nonexistent
20240,-0.578464,admin.,single,6.0,no,yes,no,cellular,aug,mon,382,0.163785,999,0,nonexistent
37244,-0.674662,blue-collar,married,3.0,no,no,no,cellular,aug,fri,99,-0.559575,999,2,failure


Let's take a look at the imputed and ordinal-encoded `education`.

**3. Nominal Feature: `job` (One-Hot Encoding with Imputation)**

- **Imputation**

In [12]:
nom_col_demo = ['job']

imputer_nom = SimpleImputer(strategy='most_frequent')
imputer_nom.fit(X_train[nom_col_demo])

X_train[nom_col_demo]= imputer_nom.fit_transform(X_train[nom_col_demo])
X_test[nom_col_demo] = imputer_nom.transform(X_test[nom_col_demo])

- **Nominal Encoding**

In [13]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

## Write your code here

X_train_onehot = onehot_encoder.fit_transform(X_train[['job']])
X_test_onehot = onehot_encoder.transform(X_test[['job']])

X_train_onehot.shape

(28831, 11)

In [14]:
X_train_onehot

array([[0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [15]:
X_train = pd.concat([X_train.reset_index(drop=True),
                               pd.DataFrame(X_train_onehot, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)

X_train

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed
0,1.441699,technician,married,5.0,no,no,no,cellular,apr,mon,376,-0.197895,999,1,failure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.867059,admin.,single,6.0,no,no,no,telephone,may,fri,239,-0.197895,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.059455,technician,married,5.0,no,yes,no,cellular,jul,wed,332,-0.559575,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.289869,services,divorced,4.0,no,no,no,cellular,may,thu,36,-0.559575,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.191122,admin.,divorced,6.0,no,no,no,telephone,may,mon,95,-0.559575,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28826,-0.482266,management,married,6.0,no,no,no,telephone,may,mon,349,-0.197895,999,0,nonexistent,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
28827,-0.482266,admin.,single,6.0,no,yes,no,cellular,nov,thu,20,1.972185,6,1,success,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28828,0.864510,self-employed,single,5.0,NaN,yes,no,telephone,nov,tue,126,0.525465,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
28829,-0.578464,blue-collar,married,3.0,no,no,no,telephone,jun,thu,761,-0.197895,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
X_test = pd.concat([X_test.reset_index(drop=True),
                               pd.DataFrame(X_test_onehot, columns=onehot_encoder.get_feature_names_out(['job']))], axis=1)

X_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed
0,-0.674662,blue-collar,married,4.0,no,yes,no,telephone,may,wed,183,0.525465,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.386068,entrepreneur,divorced,2.0,no,yes,no,cellular,may,fri,53,0.525465,999,0,nonexistent,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.482266,self-employed,single,5.0,no,yes,no,cellular,nov,mon,128,-0.197895,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,-0.963257,blue-collar,single,2.0,no,yes,no,telephone,may,fri,72,0.163785,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.634096,blue-collar,divorced,2.0,NaN,yes,yes,cellular,jul,tue,329,1.248825,999,0,nonexistent,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12352,-0.482266,services,married,4.0,no,no,no,cellular,aug,tue,63,-0.559575,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
12353,0.383518,technician,single,6.0,no,no,no,cellular,nov,thu,284,-0.559575,999,0,nonexistent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
12354,-0.578464,admin.,single,6.0,no,yes,no,cellular,aug,mon,382,0.163785,999,0,nonexistent,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12355,-0.674662,blue-collar,married,3.0,no,no,no,cellular,aug,fri,99,-0.559575,999,2,failure,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### **Exercise 1: Apply All Preprocessing & Train Logistic Regression**

Now, it's your turn to apply these preprocessing steps to *all* relevant columns and then train a Logistic Regression model.

**Instructions:**

1.  Look at the Variable Table in [this link](https://archive.ics.uci.edu/dataset/222/bank+marketing).
2. Make lists for `numerical_features`, `ordinal_features`, and `nominal_features`.
3. Preprocess the features. It is safer to make a copy of `X_train` using:
   ```
   X_train_copy = X_train.copy()
   X_test_copy = X_test.copy()
   ```
   and preprocess `X_train_copy` instead.

   **For nominal features, concat the one-hot encoded features using [`pd.concat(..., axis=1)`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) and drop the old nominal features from the dataframe.**
4. Train Logistic Regression on the preprocessed `X_train_copy` and `y_train`.
5. Evaluate the Model:
    *   Make predictions on the preprocessed `X_test_copy`.
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?


In [28]:
# --- YOUR CODE FOR EXERCISE 1 STARTS HERE ---
# Numerical
num_features = ['age', 'campaign', 'pdays', 'previous']

# Ordinal
ord_features = ['education']

# Nominal
nom_features = ['job', 'marital', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']

X_train_copy = X_train.copy()
X_test_copy = X_test.copy()

In [29]:
#Numerical Features (Standard Scaling)
scaler = StandardScaler()
X_train_copy[num_features] = scaler.fit_transform(X_train_copy[num_features])
X_test_copy[num_features] = scaler.transform(X_test_copy[num_features])

In [30]:
#Ordinal Features-> Impute & Encode
imputer_ord = SimpleImputer(strategy='most_frequent')
X_train_copy[ord_features] = imputer_ord.fit_transform(X_train_copy[ord_features])
X_test_copy[ord_features] = imputer_ord.transform(X_test_copy[ord_features])
education_categories = ['illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school','professional.course', 'university.degree', 'masters', 'doctorate']
ord_encoder = OrdinalEncoder(categories=[education_categories], handle_unknown='use_encoded_value', unknown_value=-1)
X_train_copy[ord_features] = ord_encoder.fit_transform(X_train_copy[ord_features])
X_test_copy[ord_features] = ord_encoder.transform(X_test_copy[ord_features])

In [31]:
#Nominal Features
#Imputation
imputer_nom = SimpleImputer(strategy='most_frequent')
X_train_copy[nom_features] = imputer_nom.fit_transform(X_train_copy[nom_features])
X_test_copy[nom_features] = imputer_nom.transform(X_test_copy[nom_features])
#One-Hot Encoding
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
#Fit transform
X_train_onehot = onehot_encoder.fit_transform(X_train_copy[nom_features])
X_test_onehot = onehot_encoder.transform(X_test_copy[nom_features])
#สร้าง DataFrame ของ One-Hot
new_columns = onehot_encoder.get_feature_names_out(nom_features)
# สร้าง DF ใหม่
X_train_onehot_df = pd.DataFrame(X_train_onehot, columns=new_columns, index=X_train_copy.index)
X_test_onehot_df = pd.DataFrame(X_test_onehot, columns=new_columns, index=X_test_copy.index)
#Concat
X_train_copy = pd.concat([X_train_copy, X_train_onehot_df], axis=1)
X_test_copy = pd.concat([X_test_copy, X_test_onehot_df], axis=1)
#Drop
X_train_copy.drop(columns=nom_features, inplace=True)
X_test_copy.drop(columns=nom_features, inplace=True)
# Cleanup
if 'duration' in X_train_copy.columns:
    X_train_copy.drop(columns=['duration'], inplace=True)
    X_test_copy.drop(columns=['duration'], inplace=True)

In [32]:
print(f"Final Shape X_train: {X_train_copy.shape}")
print(f"Final Shape X_test: {X_test_copy.shape}")

Final Shape X_train: (28831, 45)
Final Shape X_test: (12357, 45)


In [33]:
X_train_copy

,age,education,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_divorced,marital_married,marital_single,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
5480,-0.191760,2.0,1.225777,0.198410,-0.351189,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2728,0.384450,6.0,-0.200820,0.198410,-0.351189,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
31151,-1.248145,6.0,-0.557469,0.198410,1.663762,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
24458,0.192380,4.0,-0.557469,0.198410,-0.351189,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
39159,-0.287795,4.0,-0.200820,-5.050422,3.678713,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25102,-0.095725,4.0,0.155829,0.198410,-0.351189,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
17394,1.632904,5.0,-0.200820,0.198410,-0.351189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
21847,-1.056075,6.0,-0.557469,0.198410,-0.351189,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
22311,-0.767970,5.0,-0.557469,0.198410,-0.351189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [34]:
X_test_copy

,age,education,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_divorced,marital_married,marital_single,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
1034,0.384450,6.0,-0.557469,0.198410,-0.351189,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
29476,0.288415,5.0,-0.200820,0.198410,1.663762,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
26872,-0.479865,6.0,-0.557469,0.198410,-0.351189,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
14287,-1.344180,4.0,-0.200820,0.198410,-0.351189,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
35157,-0.095725,4.0,3.009024,0.198410,-0.351189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41108,2.305149,6.0,-0.557469,-5.055697,5.693663,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
7353,-0.383830,3.0,0.155829,0.198410,-0.351189,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
26400,-0.864005,6.0,-0.557469,0.198410,-0.351189,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
36979,0.576520,4.0,-0.557469,0.198410,-0.351189,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [35]:
log_reg = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
log_reg.fit(X_train_copy, y_train)
y_pred = log_reg.predict(X_test_copy)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.85      0.89     10970
           1       0.31      0.54      0.39      1387

    accuracy                           0.81     12357
   macro avg       0.62      0.69      0.64     12357
weighted avg       0.87      0.81      0.83     12357



===========================================================================

## Part 2: Fashion-MNIST Dataset - Image Classification

### Load Fashion-MNIST Dataset

The Fashion-MNIST dataset consists of 28x28 grayscale images of fashion items.

In [ ]:
(fm_X_train, fm_y_train), (fm_X_test, fm_y_test) = fashion_mnist.load_data()

print(f"Fashion-MNIST Train data shape: {fm_X_train.shape}")
print(f"Fashion-MNIST Train labels shape: {fm_y_train.shape}")
print(f"Fashion-MNIST Test data shape: {fm_X_test.shape}")
print(f"Fashion-MNIST Test labels shape: {fm_y_test.shape}")

In [ ]:
print(f"First image {fm_X_train[0]}")
print(f"First label {fm_y_train[0]}")

### Visualize Fashion-MNIST Images

Let's see what these images look like.

In [ ]:
fashion_mnist_class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

# Visualize the images
## Write your code here



### **Exercise 2: Preprocessing Images (Flatten and Scale)**

Images are 2D arrays (matrices of pixels) and pixel values are integers from 0-255. For Logistic Regression, we need:
*  **Flattening:** Convert each 28x28 image into a 1D array of 784 features.
*  **Scaling:** Normalize pixel values from [0, 255] to [0, 1].

**Instructions:**

1.   **Flatten:** Use the `.reshape()` method (see [documentation](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.reshape.html)). For `fm_X_train_binary` (shape `(num_samples, 28, 28)`), you want to reshape it to `(num_samples, 28*28)`.
2.  **Scale:** Divide the flattened pixel values by 255.0 to get values between 0 and 1.
3.   **Train Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
    *   Fit the model on your *processed* `fm_X_train_scaled` and `fm_y_train`.
4.   **Make Predictions:** Use `predict()` to make predictions on the *processed* `fm_X_test_scaled`.
5.   **Print Classification Report:** Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?
6.   **Visualize Misclassifications:**
    *   Find the indices in `fm_X_test_binary` where your model made incorrect predictions (i.e., `fm_y_pred != fm_y_test`).
    *   Select 5 of these misclassified images.
    *   Plot these images (using `plt.imshow`). For each image, print its true label and its predicted label.

In [ ]:
# --- YOUR CODE FOR EXERCISE 2 STARTS HERE ---





## Part 3: 20 Newsgroups Dataset - Text Classification

### Load 20 Newsgroups Dataset

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics.

In [ ]:
news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

X_train_news, y_train_news = news_train.data, news_train.target
X_test_news, y_test_news = news_test.data, news_test.target

print(f"Number of training documents: {len(X_train_news)}")
print(f"Number of test documents: {len(X_test_news)}")
print(f"Categories: {news_train.target_names}")

### Explore Sample Document

In [ ]:
# Print the first document and its class
## Write your code here



### Preprocessing: Text Vectorization Demonstration with `TfidfVectorizer`

$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$

Where:

$$
\text{TF}(t, d) = \frac{\text{number of word }t\text{ in } d}{\text{number of words in } d} \quad \text{ and } \quad
\text{IDF}(t, D) = \log\left(\frac{\text{total number of documents}}{\text{number of documents that contain word }t}\right).
$$

In [ ]:
sample_sentences = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the sample sentences
sample_vec_output_sparse = # Write your code here

sample_vec_output_dense = sample_vec_output_sparse.toarray()

print(vectorizer.vocabulary_)
print(vectorizer.get_feature_names_out())
print(sample_vec_output_dense)

### **Exercise 3: Apply TF-IDF Vectorization to Full Dataset**

Now, apply `TfidfVectorizer` to the actual training and testing datasets for the 20 Newsgroups classification task.

**Instructions:**

1.  **Initialize `TfidfVectorizer`:**
    *   Initialize `TfidfVectorizer`. Use `stop_words='english'` to remove common words.
2.  **Fit and Transform Training Data:**
    *   Call `fit_transform()` on `X_train_news` to learn the vocabulary and transform the training text into TF-IDF features. Store the result in `X_train_vec`.
3.  **Transform Test Data:**
    *   Call `transform()` on `X_test_news` using the *already fitted* vectorizer. Store the result in `X_test_vec`. **Crucially, do not call `fit_transform()` on the test data!** This would cause data leakage.
4.  **Initialize Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
5.  **Train the Model:**
    *   Fit the model on your `X_train_vec` and `y_train_news`.
6.  **Make Predictions:**
    *   Make predictions using `predict()` on the `X_test_vec`.
7.  **Evaluate the Model:**
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?

In [ ]:
# --- YOUR CODE FOR EXERCISE 3 STARTS HERE ---


